In [1]:
# cite simcse
# cite sentence-transformers https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
import pandas as pd
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

## My Dataset

In [2]:
label = "final_pred_modified"

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [4]:
#df = pd.read_excel("data/survey_data/labelled_data/sat_data_combined_vertical_pred_final.xlsx")
df = pd.read_excel("../data/survey_data/labelled_data/data_labelled.xlsx")

In [5]:
df[label] = df[label].astype(int)

In [6]:
X = df['response']
y = df[label]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=0)

In [8]:
texts = list(X_train)
texts_test = list(X_test)
texts_val = list(X_val)

In [9]:
df[label].value_counts()

2    871
0    824
1    673
Name: final_pred_modified, dtype: int64

In [10]:
X_train

526     Are you aware of on-goings within your local c...
92      Congratulations! You have shown foresighted co...
963     I would like to emphasise that the purpose of ...
909     Congratulations! You have shown tender compass...
2242    This is a fantastic start but bear in mind tha...
                              ...                        
1033     As a first step : shall we continue with ESA?...
1731    I would like to emphasize that the purpose of ...
763     Think about how you plan to achieve this. When...
835                        How are you feeling right now?
1653    Let's talk a little bit more about you - Do yo...
Name: response, Length: 1657, dtype: object

## Model prediction

In [11]:

#tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
#model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
#tokenizer = AutoTokenizer.from_pretrained("roberta-base")
#model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=3)
#tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")
#model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment", num_labels=3)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [12]:
from torch.utils.data import Dataset, DataLoader
class EmpatheticDataset(Dataset):
    def __init__(self, X, y, with_label=True):
        self.X = X.reset_index()['response']
        self.y = y.reset_index()[label]
        self.text = list(X.values)
        encoded_input = tokenizer(self.text, padding=True, truncation=True, return_tensors="pt")
        encoded_input['text'] = self.text
        encoded_input['label'] = torch.tensor(y.values)
        if with_label:
            try:
                self.new_output = [{'label': label, 'text': text,'input_ids': input_ids, \
                           'token_type_ids': token_type_ids, 'attention_mask': attention_mask} \
                           for label, text, input_ids, token_type_ids, attention_mask in zip(encoded_input['label'], \
                                                                              encoded_input['text'], \
                                                                              encoded_input['input_ids'], \
                                                                              encoded_input['token_type_ids'], \
                                                                              encoded_input['attention_mask'])]
            except:
                self.new_output = [{'label': label, 'text': text,'input_ids': input_ids, \
                           'attention_mask': attention_mask} \
                           for label, text, input_ids, attention_mask in zip(encoded_input['label'], \
                                                                              encoded_input['text'], \
                                                                              encoded_input['input_ids'], \
                                                                              encoded_input['attention_mask'])]
        else:
            try:
                self.new_output = [{'text': text,'input_ids': input_ids, \
                           'token_type_ids': token_type_ids, 'attention_mask': attention_mask} \
                           for text, input_ids, token_type_ids, attention_mask in zip(encoded_input['text'], \
                                                                              encoded_input['input_ids'], \
                                                                              encoded_input['token_type_ids'], \
                                                                              encoded_input['attention_mask'])]
            except:
                self.new_output = [{'text': text,'input_ids': input_ids, \
                           'attention_mask': attention_mask} \
                           for text, input_ids, attention_mask in zip(encoded_input['text'], \
                                                                              encoded_input['input_ids'], \
                                                                              encoded_input['attention_mask'])]
            
        
 
    def __len__(self):
        return len(self.X)
   
    def __getitem__(self,idx):
        try:
            return self.new_output[idx]
        except:
            print(idx)
            print(self.new_output[idx - 1])
            print(self.new_output[idx])
        #return self.X[idx], self.y[idx]
        #features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask']

In [13]:
train_data = EmpatheticDataset(X_train, y_train)
test_data = EmpatheticDataset(X_test, y_test)
val_data = EmpatheticDataset(X_val, y_val)

In [14]:
#train_dataloader = DataLoader(train_data, shuffle=True, batch_size=8)
#test_dataloader = DataLoader(test_data, batch_size=8)

In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [16]:
from datasets import load_metric

metric = load_metric("accuracy")

In [17]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,
)

In [20]:
trainer.train()

/Users/weijiechua/miniforge3/envs/gen/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1657
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 624
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.904064,0.566197
2,No log,0.598452,0.763380
3,0.768700,0.513781,0.816901


***** Running Evaluation *****
  Num examples = 355
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 355
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 355
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertFo

TrainOutput(global_step=624, training_loss=0.7080848583808312, metrics={'train_runtime': 1119.227, 'train_samples_per_second': 4.441, 'train_steps_per_second': 0.558, 'total_flos': 380630045174526.0, 'train_loss': 0.7080848583808312, 'epoch': 3.0})

In [21]:
#save_path = "bert-base-uncased_full_data.pt"

In [22]:
#torch.save(model.state_dict(), save_path)

In [17]:
save_path = "../models/bert-base-uncased_full_data.pt"

In [27]:
model2 = torch.load(save_path)

## Loading Pre-trained model

In [12]:
model2 = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [18]:
model2.load_state_dict(torch.load(save_path, map_location="cpu"))

FileNotFoundError: [Errno 2] No such file or directory: '../models/bert-base-uncased_full_data.pt'

In [37]:
trainer2 = Trainer(
    model=model2,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,
)

In [38]:
y_pred_hf = trainer2.predict(test_data)

***** Running Prediction *****
  Num examples = 356
  Batch size = 8
The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.


# Inference

In [120]:
y_pred_hf = trainer.predict(test_data)

***** Running Prediction *****
  Num examples = 356
  Batch size = 8
The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.


In [39]:
y_pred_hf

PredictionOutput(predictions=array([[-0.768334  ,  2.197935  , -1.7008576 ],
       [ 2.3556778 , -0.7962154 , -1.963906  ],
       [-1.781242  , -1.0795318 ,  3.0932853 ],
       ...,
       [-1.9587072 ,  0.31000274,  1.3631008 ],
       [ 2.5384872 , -1.3550705 , -1.5202875 ],
       [-1.3955543 ,  1.7085536 , -0.654751  ]], dtype=float32), label_ids=array([1, 0, 2, 0, 2, 2, 0, 0, 1, 0, 2, 2, 1, 1, 0, 1, 0, 0, 1, 2, 0, 0,
       1, 2, 1, 0, 2, 0, 1, 2, 1, 1, 0, 2, 1, 2, 2, 1, 2, 2, 0, 1, 0, 2,
       0, 0, 1, 2, 0, 2, 2, 0, 0, 2, 2, 1, 1, 0, 2, 0, 2, 2, 0, 0, 0, 2,
       2, 1, 1, 0, 2, 2, 0, 2, 2, 1, 1, 1, 0, 0, 2, 1, 0, 0, 0, 2, 2, 0,
       1, 0, 2, 0, 0, 1, 1, 0, 1, 2, 2, 0, 1, 2, 2, 1, 2, 0, 1, 0, 0, 1,
       0, 1, 2, 2, 2, 2, 2, 1, 2, 2, 0, 1, 0, 0, 0, 2, 2, 1, 0, 1, 1, 2,
       1, 2, 2, 1, 0, 0, 1, 2, 1, 2, 0, 1, 1, 1, 0, 1, 1, 0, 2, 2, 1, 2,
       2, 2, 0, 1, 0, 0, 0, 1, 1, 0, 2, 2, 1, 0, 2, 1, 2, 1, 0, 0, 0, 1,
       1, 2, 0, 2, 2, 0, 1, 2, 2, 2, 0, 0, 2, 0, 1, 0, 0, 1,

In [40]:
y_pred_dl = np.argmax(y_pred_hf[0], axis=1)

In [41]:
#y_pred_dl

In [42]:
y_pred_dl

array([1, 0, 2, 2, 2, 1, 1, 0, 1, 0, 2, 2, 1, 2, 0, 2, 2, 0, 2, 0, 0, 0,
       1, 2, 2, 0, 2, 0, 1, 2, 1, 1, 0, 2, 1, 2, 2, 1, 2, 2, 0, 1, 0, 2,
       0, 0, 1, 2, 0, 0, 2, 0, 0, 2, 2, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 2,
       2, 1, 1, 0, 1, 2, 0, 2, 2, 1, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 2, 0,
       1, 1, 2, 0, 0, 1, 1, 0, 1, 2, 2, 0, 2, 2, 2, 1, 2, 0, 1, 0, 0, 1,
       0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 1, 0, 2, 0, 2, 0, 0, 0, 2, 1, 2,
       1, 2, 2, 1, 0, 0, 1, 2, 1, 2, 0, 1, 1, 0, 2, 1, 0, 0, 0, 2, 2, 2,
       2, 2, 0, 1, 0, 0, 0, 1, 1, 0, 2, 2, 1, 1, 1, 1, 2, 0, 0, 0, 0, 2,
       1, 2, 0, 1, 2, 0, 1, 2, 2, 1, 1, 0, 1, 0, 1, 0, 2, 1, 0, 0, 2, 0,
       2, 2, 0, 0, 0, 2, 2, 0, 1, 1, 2, 2, 2, 2, 0, 0, 2, 0, 1, 0, 0, 0,
       2, 0, 2, 0, 2, 1, 2, 0, 0, 0, 1, 0, 2, 1, 0, 0, 0, 0, 2, 2, 1, 2,
       0, 2, 0, 0, 2, 1, 2, 1, 0, 2, 1, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 0,
       2, 2, 2, 1, 0, 2, 0, 1, 2, 1, 0, 0, 2, 2, 2, 0, 2, 2, 1, 2, 1, 2,
       1, 2, 2, 1, 0, 2, 1, 2, 2, 2, 2, 2, 0, 1, 2,

In [43]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_dl))

              precision    recall  f1-score   support

           0       0.86      0.87      0.87       127
           1       0.80      0.67      0.73       105
           2       0.78      0.88      0.83       124

    accuracy                           0.81       356
   macro avg       0.81      0.81      0.81       356
weighted avg       0.81      0.81      0.81       356

